In [ ]:
!pip install --upgrade pydrive2

!pip install snowflake-connector-python pandas -q


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.


In [ ]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [ ]:
SNOWFLAKE_CONFIG = {
    'user': 'salvador',
    'password': 'qiwpebJefroczizfa6',
    'account': 'myfsmos-wlb40068',
    'warehouse': 'COMPUTE_WH',
    'database': 'CARD_TRANSACTION',
    'schema': 'SILVER'
}

conn = snowflake.connector.connect(
    user=SNOWFLAKE_CONFIG['user'],
    password=SNOWFLAKE_CONFIG['password'],
    account=SNOWFLAKE_CONFIG['account'],
    warehouse=SNOWFLAKE_CONFIG['warehouse'],
    database=SNOWFLAKE_CONFIG['database'],
    schema=SNOWFLAKE_CONFIG['schema']
)

cursor = conn.cursor()
print("Conexion establecida")


Conexion establecida


In [ ]:
#Extraccion

df_merchants = pd.read_sql("SELECT * FROM SILVER.DIM_MERCHANTS",conn)
df_transactions = pd.read_sql("SELECT * FROM SILVER.TRANSACTIONS_STRUCTURED",conn)

print(f"Tabla transactions: {len(df_transactions):,}")
print(f"Tabla merchants: {len(df_merchants):,}")

/tmp/ipython-input-1085618577.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_merchants = pd.read_sql("SELECT * FROM SILVER.DIM_MERCHANTS",conn)
/tmp/ipython-input-1085618577.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_transactions = pd.read_sql("SELECT * FROM SILVER.TRANSACTIONS_STRUCTURED",conn)


Tabla transactions: 1,296,675
Tabla merchants: 700


In [ ]:
print("Columnas en Transactions:", df_transactions.columns.tolist())
print("-" * 30)
print("Columnas en Merchants:", df_merchants.columns.tolist())

Columnas en Transactions: ['TRANS_NUM', 'CC_NUM', 'TRANSACTION_TIMESTAMP', 'AMOUNT', 'MERCHANT_CLEAN', 'CATEGORY_CLEAN', 'IS_FRAUD', 'TRANSACTION_TYPE']
------------------------------
Columnas en Merchants: ['MERCHANT_CLEAN', 'CATEGORY_CLEAN', 'TRANSACTION_COUNT', 'AVG_TRANSACTION_AMOUNT']


In [ ]:

df_gold = df_transactions.merge(
    df_merchants,
    on='MERCHANT_CLEAN',
    how='left'
)

print(f"Consolidación completa: {len(df_gold):,} registros")

Consolidación completa: 1,321,301 registros


In [ ]:
cursor.execute("CREATE SCHEMA IF NOT EXISTS GOLD")

print("Esquema GOLD listo")

Esquema GOLD listo


In [ ]:
success, nchunks, nrows, _ = write_pandas(
    conn=conn,
    df=df_gold,
    table_name='TRANSACTION_GOLD',
    schema='GOLD',
    database=SNOWFLAKE_CONFIG['database'],
    auto_create_table=True,
    overwrite=True
)

print(f" Datos cargados: {nrows:,} registros")

✓ Datos cargados: 1,321,301 registros
